In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt

#Order of emotion in one_hot: Anger, Disgust, Fear, Happy, Neutral, Sad
emotions = np.array(['ANG','DIS','FEA','HAP','NEU','SAD'])

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
cd '/content/gdrive/My Drive/Colab Notebooks/Voice_CNN/'

/content/gdrive/My Drive/Colab Notebooks/Voice_CNN


In [ ]:
audiolength = np.load('audiolength.npy')
one_hot = np.load('one_hot.npy')
[min_a, max_a,max_length,entry] = np.load('stats.npy')
all_mels_padded = np.load('all_mels_padded.npy')

#Train-test split

In [ ]:
split = np.floor(entry*0.9)
Input_Shape = [256,int(max_length),1]

In [ ]:
all_mels_padded = np.array(all_mels_padded)
all_mels_padded.min()

0.0

In [ ]:
all_mels_padded = all_mels_padded.reshape([-1]+Input_Shape)
mels_train = all_mels_padded[:int(split)]
mels_test = all_mels_padded[int(split):]

#Model 1

In [ ]:
from tensorflow.python.ops.math_ops import xdivy
from tensorflow.keras.layers import *


tf.keras.backend.clear_session()

inputs = tf.keras.Input(shape = Input_Shape)
conv_1 = Conv2D(
    filters = 4, kernel_size = (3,3),
    strides = (1,1), padding = 'same',
    activation = 'relu'
)(inputs)
maxpool_1 = MaxPooling2D(
    pool_size=(2, 2),
    strides=(2,2),
    padding = 'valid'
)(conv_1)
conv_2 = Conv2D(
    filters = 8, kernel_size = (3,3),
    strides = (1,1), padding = 'same',
    activation = 'relu'
)(maxpool_1)
maxpool_2 = MaxPooling2D(
    pool_size=(2, 2),
    strides=(2,2),
    padding = 'valid'
)(conv_2)
conv_3 = Conv2D(
    filters = 16, kernel_size = (3,3),
    strides = (1,1), padding = 'same',
    activation = 'relu'
)(maxpool_2)
maxpool_3 = MaxPooling2D(
    pool_size=(2, 2),
    strides=(2,2),
    padding = 'valid'
)(conv_3)
flatten = Flatten()(maxpool_3)
outputs = Dense(units=len(emotions),activation='softmax')(flatten)

model = tf.keras.Model(
    inputs = inputs, outputs = outputs,
    name = 'cnn-1'
)

optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
)

model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy']
)

model.summary()

Model: "cnn-1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 79, 1)]      0         
                                                                 
 conv2d (Conv2D)             (None, 256, 79, 4)        40        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 39, 4)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 39, 8)        296       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 64, 19, 8)        0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 19, 16)        1168  

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

batch_size = 64
epochs = 500


es = EarlyStopping(
    monitor='val_accuracy',
    min_delta = 0,
    patience = 500,
    verbose = 1,
    mode = 'max',
    restore_best_weights = True
)

history = model.fit(mels_train, one_hot[:int(split)], epochs=epochs, batch_size=batch_size, 
                    validation_split=0.3, callbacks=[es])




In [ ]:
res = pd.DataFrame(history.history)
res[['loss','val_loss']].plot()

In [ ]:
res[['accuracy','val_accuracy']].plot()

In [ ]:
scores = model.evaluate(mels_test, one_hot[int(split):], verbose=1)
names = model.metrics_names

for i in range(len(scores)):
  print(f'{names[i]}: {scores[i]:.4f}')

In [ ]:
model.save('/content/gdrive/MyDrive/Colab Notebooks/Voice_CNN/6_2_2023_model')

In [ ]:
model.load_weights('/content/gdrive/MyDrive/Colab Notebooks/Voice_CNN/6_2_2023_model_weights')